In [ ]:
# read upgrade_dataset.json
# take url and donload with pytube library, save it into a music .wav file in data/genres_original/{genre}/{genre}.upgrade.{index}.wav, shorten t to a multiple of 30 seconds
# my json is like this {"blues": [{"title": "The Thrill Is Gone - B.B. King", "url": "https://www.youtube.com/watch?v=4fk2prKnYnI"}, ... ], ...}

In [21]:
%pip install pytube pydub requests beautifulsoup4
%cd /tf/notebooks/notebooks

Note: you may need to restart the kernel to use updated packages.
/tf/notebooks/notebooks


/usr/local/lib/python3.11/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [27]:
import requests
from pytube import Search
from bs4 import BeautifulSoup

def check_valid_url(url):
    try:
        response = requests.get(url)

        if response.status_code != 200:
            return False
        
        # Parse the HTML content of the webpage
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the div with id 'columns' and classes 'style-scope' and 'ytd-watch-flexy'
        div_element = soup.find('div', id='columns', class_='style-scope ytd-watch-flexy')

        if not div_element:
            print("div_element not found")
            return False

        return not div_element.get('hidden')

    except requests.RequestException:
        return False
    
def trouver_url_video(musique):
    try:
        recherche = Search(musique)
        video = recherche.results[0]  # Récupère la première vidéo de la liste des résultats
        return video.watch_url
    except Exception as e:
        print("Erreur lors de la recherche :", e)
        return None

def replace_invalid_urls(data):
    for genre, songs in data.items():
        for song in songs:
            if not check_valid_url(song['url']):
                print(f"URL invalide pour {song['title']} : {song['url']}")
                # new_url = trouver_url_video(song['title'])
                # if new_url is not None:
                #     print(f"Remplacement de l'URL invalide {song['url']} par {new_url}")
                #     song['url'] = new_url
                # else:
                #     print(f"Impossible de trouver une URL valide pour {song['title']}")
            else:
                print(f"URL valide pour {song['title']} : {song['url']}")
    return data

import json

with open('upgrade_dataset.json', 'r') as f:
    data = json.load(f)

print("Remplacement des URLs invalides...")
data_replace = replace_invalid_urls(data)
print("Remplacement terminé !")

with open('upgrade_dataset.json', 'w') as f:
    json.dump(data_replace, f)

Remplacement des URLs invalides...
div_element not found
URL invalide pour Need Your Love So Bad - Fleetwood Mac : https://www.youtube.com/watch?v=RtmW2ek7WkQ
div_element not found
URL invalide pour Mannish Boy - Muddy Waters : https://www.youtube.com/watch?v=nU5uDozoSSM
div_element not found
URL invalide pour Fever - Little Willie John : https://www.youtube.com/watch?v=jLoIe3W4R6w
div_element not found
URL invalide pour Going Down - Freddie King : https://www.youtube.com/watch?v=8qLlvdXaRs0
div_element not found
URL invalide pour I'm Tore Down - Freddie King : https://www.youtube.com/watch?v=pOLYyM_i8Hg
div_element not found
URL invalide pour Baby Please Don't Go - Big Joe Williams : https://www.youtube.com/watch?v=Y65Q9-vJ1og
div_element not found
URL invalide pour Big Boss Man - Jimmy Reed : https://www.youtube.com/watch?v=8JL7AKZWd6k
div_element not found
URL invalide pour Rollin' Stone - Muddy Waters : https://www.youtube.com/watch?v=0cfYAwTPt_8
div_element not found
URL invalide 

KeyboardInterrupt: 

In [15]:
import json
import os
from pytube import YouTube
from pydub import AudioSegment

# Load data from the JSON file
with open('upgrade_dataset.json', 'r') as file:
    data = json.load(file)

# Define a function to download and process the audio
def download_and_process_audio(genre, title, url, index):
    try:
        yt = YouTube(url)
        stream = yt.streams.filter(only_audio=True).first()
        
        # if mp4 is not already downloaded, do it, otherwise skip
        filename = f"data/genres_original/{genre}/{genre}.upgrade.{index}.wav"

        if not os.path.exists(filename):
            # Download the video as audio
            stream.download(filename=f"data/temp/{genre}_{index}.mp4")
            
            # Convert the downloaded file to a WAV audio file
            audio = AudioSegment.from_file(f"data/temp/{genre}_{index}.mp4", format="mp4")
            
            # Ensure duration is a multiple of 30 seconds
            duration = len(audio)
            duration = duration - (duration % (30 * 1000))  # Round down to the nearest multiple of 30 seconds
            audio = audio[:duration]  # Trim audio to the desired duration
            
            # Export the audio as a WAV file
            audio.export(filename, format="wav")
            
            print(f"Downloaded and processed: {title}")
        else:
            print(f"Already downloaded: {title}")
            
    except Exception as e:
        print(f"Error processing {title}: {e}")

# Process each entry in the JSON data
for genre, songs in data.items():
    genre_dir = f"data/genres_original/{genre}/"
    if not os.path.exists(genre_dir):
        os.makedirs(genre_dir)
    
    for index, song in enumerate(songs):
        title = song['title']
        url = song['url']
        download_and_process_audio(genre, title, url, index)


Already downloaded: Need Your Love So Bad - Fleetwood Mac
Error processing Mannish Boy - Muddy Waters: nU5uDozoSSM is unavailable
Error processing Fever - Little Willie John: jLoIe3W4R6w is unavailable
Error processing Going Down - Freddie King: 8qLlvdXaRs0 is unavailable
Error processing I'm Tore Down - Freddie King: pOLYyM_i8Hg is unavailable
Error processing Baby Please Don't Go - Big Joe Williams: Y65Q9-vJ1og is unavailable
Error processing Big Boss Man - Jimmy Reed: 8JL7AKZWd6k is unavailable
Error processing Rollin' Stone - Muddy Waters: 0cfYAwTPt_8 is unavailable
Already downloaded: Smokestack Lightnin' - Howlin' Wolf
Error processing Good Morning Little Schoolgirl - Sonny Boy Williamson: iKZ7GZsOxVE is unavailable
Already downloaded: Ludwig van Beethoven - Moonlight Sonata
Already downloaded: Johann Sebastian Bach - Cello Suite No. 1
Error processing Wolfgang Amadeus Mozart - Requiem in D minor: sPlhKP0nZII is unavailable
Error processing Pyotr Ilyich Tchaikovsky - Swan Lake: z